In [1]:
!pip install seaborn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 7.3 MB/s eta 0:00:00a 0:00:01


In [2]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 3.0 MB/s eta 0:00:00
  Using cached termcolor-2.4.0-py3-none-any.whl.metadata (6.1 kB)
INFO: pip is looking at multiple versions of classla to determine which version is compatible with other requirements. This could take a while.
  Using cached wheel-0.43.0-py3-none-any.whl.metadata (2.2 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 45.0 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 59.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 74.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.1/117.1 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 17.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.6/202.6 kB 21.3

In [1]:
import pandas as pd
import pickle
from datasets import Dataset
import torch
from datasets import DatasetDict
from transformers import BertForSequenceClassification, BertTokenizer
from transformers import DataCollatorWithPadding
import evaluate
from transformers import TrainingArguments, Trainer
import numpy as np

/home/zan/Faks/NLP/ul-fri-nlp-course-project-azb/.testvenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
train_df = pd.read_pickle("train_dataset")
test_df = pd.read_pickle("test_dataset")
val_df = pd.read_pickle("val_dataset")

In [6]:
dataset = DatasetDict({
    'train': Dataset.from_pandas(train_df),
    'test': Dataset.from_pandas(test_df),
    'unsupervised': Dataset.from_pandas(val_df)
})

In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 427
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 92
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 92
    })
})

In [8]:
# define preprocess function
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [26]:
tokenizer = BertTokenizer.from_pretrained("bert-base-cased")

/home/zan/Faks/NLP/ul-fri-nlp-course-project-azb/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [27]:
tokenized_data = dataset.map(preprocess_function, batched=True, batch_size=100,  load_from_cache_file=True)

Map: 100%|██████████| 92/92 [00:00<00:00, 2916.89 examples/s]


In [28]:
tokenized_data['train'][0].keys()

dict_keys(['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'])

In [29]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [30]:
accuracy = evaluate.load("accuracy")

In [31]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [32]:
train_df['label'].unique()

array([4, 5, 6, 0, 3, 1, 2], dtype=int32)

In [33]:
model = BertForSequenceClassification.from_pretrained("bert-base-cased", num_labels=7)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [34]:
BATCH_SIZE = 8
EPOCHS = 3
training_args = TrainingArguments(
    output_dir="./bert_runs_2",
    learning_rate=2e-5,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=EPOCHS,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

/home/zan/Faks/NLP/ul-fri-nlp-course-project-azb/.venv/lib/python3.10/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [35]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [36]:
torch.cuda.empty_cache()

In [37]:
trainer.train()

 33%|███▎      | 54/162 [01:52<02:50,  1.58s/it]

{'eval_loss': 1.084995150566101, 'eval_accuracy': 0.6630434782608695, 'eval_runtime': 8.7663, 'eval_samples_per_second': 10.495, 'eval_steps_per_second': 1.369, 'epoch': 1.0}


 67%|██████▋   | 108/162 [04:02<01:00,  1.11s/it]

{'eval_loss': 0.8796781301498413, 'eval_accuracy': 0.75, 'eval_runtime': 7.1179, 'eval_samples_per_second': 12.925, 'eval_steps_per_second': 1.686, 'epoch': 2.0}


100%|██████████| 162/162 [06:28<00:00,  1.81s/it]

{'eval_loss': 0.8326335549354553, 'eval_accuracy': 0.7608695652173914, 'eval_runtime': 7.5144, 'eval_samples_per_second': 12.243, 'eval_steps_per_second': 1.597, 'epoch': 3.0}


100%|██████████| 162/162 [06:30<00:00,  2.41s/it]

{'train_runtime': 390.5799, 'train_samples_per_second': 3.28, 'train_steps_per_second': 0.415, 'train_loss': 1.1228936277789834, 'epoch': 3.0}


TrainOutput(global_step=162, training_loss=1.1228936277789834, metrics={'train_runtime': 390.5799, 'train_samples_per_second': 3.28, 'train_steps_per_second': 0.415, 'total_flos': 50810978521350.0, 'train_loss': 1.1228936277789834, 'epoch': 3.0})

In [38]:
trainer.evaluate()

100%|██████████| 12/12 [00:07<00:00,  1.66it/s]


{'eval_loss': 0.8326335549354553,
 'eval_accuracy': 0.7608695652173914,
 'eval_runtime': 8.0426,
 'eval_samples_per_second': 11.439,
 'eval_steps_per_second': 1.492,
 'epoch': 3.0}

In [39]:
trainer.predict(tokenized_data['unsupervised'])

100%|██████████| 12/12 [00:06<00:00,  1.76it/s]


PredictionOutput(predictions=array([[ 1.67214051e-01, -1.33009374e-01, -1.81199908e+00,
         1.89175606e-02,  2.44136244e-01,  1.70999920e+00,
        -8.51679325e-01],
       [ 8.74340653e-01, -1.24298096e+00, -1.34070504e+00,
         9.01062340e-02,  7.50245810e-01,  4.83256429e-01,
        -1.93972498e-01],
       [-8.24218392e-01, -5.16457140e-01, -1.14895308e+00,
        -6.75278664e-01,  4.27939320e+00, -6.05731726e-01,
        -7.36461639e-01],
       [-3.17297801e-02, -7.38250732e-01, -1.34872317e+00,
         7.40337491e-01, -4.16370153e-01,  1.69975173e+00,
         3.23330909e-01],
       [ 1.03139445e-01, -6.49575531e-01, -2.02592158e+00,
         6.50595725e-02,  1.45530283e+00,  1.32096744e+00,
        -9.62691307e-01],
       [ 1.91293135e-01, -6.71108961e-01, -1.36205482e+00,
        -6.35987878e-01,  3.56109667e+00, -6.62697911e-01,
        -9.05435622e-01],
       [-6.31930351e-01, -3.81327540e-01, -1.31442297e+00,
        -7.21978188e-01,  4.21775818e+00, -7.123

In [20]:
training_args = TrainingArguments(
    output_dir="./bert_runs_2",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

In [21]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [22]:
torch.cuda.empty_cache()

In [23]:
trainer.train()

C:\Users\Ana\Desktop\local playground NLP\nlp\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.874890,0.750000
2,No log,0.911776,0.717391
3,No log,0.932546,0.717391


TrainOutput(global_step=162, training_loss=0.4996317168812693, metrics={'train_runtime': 79.2057, 'train_samples_per_second': 16.173, 'train_steps_per_second': 2.045, 'total_flos': 52204193546700.0, 'train_loss': 0.4996317168812693, 'epoch': 3.0})

In [24]:
trainer.evaluate()

{'eval_loss': 0.8748903870582581,
 'eval_accuracy': 0.75,
 'eval_runtime': 1.498,
 'eval_samples_per_second': 61.415,
 'eval_steps_per_second': 8.011,
 'epoch': 3.0}

In [25]:
trainer.save_model('./baseline_bert_3_8_base_cased')

In [26]:
trainer.predict(tokenized_data['unsupervised'])

PredictionOutput(predictions=array([[ 0.34760496, -1.5118971 , -1.6656618 ,  0.24111393,  0.25556564,
         2.2733667 , -0.8934638 ],
       [ 1.2248296 , -0.9374818 , -1.2601331 ,  0.6985131 , -0.5656925 ,
         0.4706643 ,  0.56793666],
       [-0.91883755, -0.17711115, -1.5839896 , -1.5359768 ,  4.4879656 ,
        -1.1382575 , -1.2468565 ],
       [ 1.3300622 , -0.83188516, -1.2226257 ,  0.8510202 , -0.732101  ,
         0.30388665,  0.7443369 ],
       [-0.16029456, -1.2254049 , -1.8191245 , -0.27321234,  1.4364688 ,
         2.0361614 , -1.116059  ],
       [-0.30644032, -0.30896565, -2.0040507 , -1.1752982 ,  4.0050015 ,
        -1.08024   , -0.802896  ],
       [-0.7669351 , -0.23882994, -1.6607765 , -1.4821094 ,  4.435495  ,
        -1.1536027 , -1.2831417 ],
       [ 0.47533256, -1.3302677 , -1.2625556 ,  0.5623014 , -0.74744153,
         2.447754  , -0.5985521 ],
       [-1.0198313 , -0.06109704, -1.5908053 , -1.4989396 ,  4.3645525 ,
        -1.097253  , -1.2201146 ],